In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
from ps.partner_selection import PartnerSelection
from ps.ps_utils import get_sector_data
import pandas as pd
import pyvinecopulib as pv
import numpy as np
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
from copula.c_vine_model import CVineModel


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load returns of selected quadruples
df = pd.read_csv('./data/data/data.csv', parse_dates=True, index_col='Date').dropna()
df_training = df['2017-01-01':'2017-12-31'] #Taking 12 month data as mentioned in the paper
ps = PartnerSelection(df_training)

constituents = pd.read_csv('./data/data/constituents-detailed.csv', index_col='Symbol')

q = pd.read_csv('./data/quadruples.csv').to_numpy()
all_returns = ps.get_selected_pair_returns(q)

In [ ]:
# Get returns for one quadruple and convert to pseudo observations (normalised ranked)
quadruple = 1
training_returns = all_returns[quadruple].to_numpy()[1:]
u = pv.to_pseudo_obs(training_returns)
u = pd.DataFrame(u, columns=q[quadruple])

In [ ]:
c = CVineModel('~/R/x86_64-pc-linux-gnu-library/4.1/VineCopula/')
family = [1, 2, 3, 4, 5, 6, 7, 9, 10,
          13, 16, 20, 23, 24, 26, 27, 29, 30,
          33, 34, 36, 39, 40, 
          104, 114, 124, 134,
          204, 214, 224, 234]
c.fit(u, family)
print(c.summary())

In [ ]:

df_testing = df['2017-01-01':'2018-12-31'] #Taking 12 month data as mentioned in the paper
ps = PartnerSelection(df_testing)

constituents = pd.read_csv('./data/data/constituents-detailed.csv', index_col='Symbol')

returns = ps.get_selected_pair_returns(q)
testing_returns = returns[quadruple].to_numpy()[1:]
u2 = pv.to_pseudo_obs(testing_returns)
u2 = pd.DataFrame(u2, columns=q[quadruple])


In [ ]:
p = 4
for i in range(p,1,-1):
    for j in range(i-1,0,-1):
        print(f"({i-2}, {j-1}) = ({i-1}, {j-1}), ({i-1}, {i-1})")

In [ ]:
start = 0
stop = 249
p = []
while stop < u2.shape[0]:
    group = u2.iloc[start:stop]
    c.fit(group, family)
    p.append(c.predict(group.iloc[-1:]))
    
    start += 1
    stop += 1

print(p)